In [ ]:
"""
This notebook finds black/redacted areas and segments them (green is default), such that digital redaction may be performed downstream.
"""

In [1]:
from PIL import Image
from scipy.ndimage import gaussian_filter
from skimage.measure import label, regionprops
from skimage.segmentation import expand_labels
from typing import Tuple
from IPython.display import display
import plotly.graph_objects as go
import tqdm
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
ECG_FOLDER = '../../data/ecg_ahus_phone'
SAVE_FOLDER = '../../data/redacted_ecg_ahus_phone'

ECG_FOLDER = '/home/datasets/ecg-digitization/digital_redaction/agnar_phone_all'
SAVE_FOLDER = '/home/datasets/ecg-digitization/digital_redaction/agnar_phone_all_redacted'

elias_params =  [2e-2, 1.5e-2, 1e-2, 9e-3, 8e-3, 7e-3, 6e-3, 5e-3, 4e-3, 3e-3, 2e-3, 1e-3]
agnar_params =  [1.3e-2, 1.2e-2, 1.1e-2, 1e-2, 9e-3, 8e-3, 7e-3, 6e-3, 5e-3, 4e-3, 3e-3, 2e-3, 1e-3]
REDACTED_AREA_PROPORTIONS = agnar_params

DISPLAY = False

In [ ]:
def redact_image(
        image_path: str,
        min_redacted_area_proportion: float = 1e-2,
        redacted_factor: float = 1.2,
        expand_redact_pixels: int = 10,
        display: bool = False
    ) -> Tuple[np.ndarray, np.ndarray]:


    img = Image.open(image_path)
    img_gray = img.convert('L')
    img_np = np.array(img_gray)

    redacted_treshold_grayscale = np.percentile(img_np.flatten(), min_redacted_area_proportion*100)

    redacted_treshold_grayscale = redacted_treshold_grayscale * redacted_factor
    print(redacted_treshold_grayscale)
    print(f'Redacted threshold grayscale: {redacted_treshold_grayscale}')


    if display:
        plt.imshow(img_np)

    img_blurred = gaussian_filter(img_np, sigma=3)
    redacted = img_blurred < redacted_treshold_grayscale

    if display:
        plt.figure()
        plt.imshow(redacted, cmap='gray')
        plt.show()


    labeled = label(redacted)
    regions = regionprops(labeled)

    regions_filtered = []
    ids = []

    min_area_proportion = 2e-4
    min_area_perimeter_coefficient = 10
    for region in regions:
        if region.area / img_np.size < min_area_proportion:
            continue
        if region.area / region.perimeter < min_area_perimeter_coefficient:
            continue

        regions_filtered.append(region)
        ids.append(region.label)


    areas = [region.area for region in regions_filtered]
    perimeters = [region.perimeter for region in regions_filtered]

    if display:
        plt.plot(areas, perimeters, 'o')
        plt.xlabel('Area')
        plt.ylabel('Perimeter')
        plt.title('Area vs Perimeter of Regions')
        plt.grid()

    regions_after_filtering = np.isin(labeled, ids)

    if display:
        s = int(np.sqrt(min_area_proportion * img_np.size)//2)
        print(f'Showing {s} pixels around the redacted regions')
        regions_after_filtering[regions_after_filtering.shape[0]//2-s:regions_after_filtering.shape[0]//2+s, regions_after_filtering.shape[1]//2-s:regions_after_filtering.shape[1]//2+s] = 255
        plt.figure()
        plt.imshow(regions_after_filtering, cmap='gray')

    regions_after_filtering = expand_labels(regions_after_filtering, distance=expand_redact_pixels)

    if display:
        plt.figure()
        plt.imshow(regions_after_filtering, cmap='gray')

    img_redacted = np.array(img)
    img_redacted[regions_after_filtering > 0] = np.array([0, 255, 0])

    # Plot summary of pixel values.
    if display:
        plt.figure()
        plt.hist(img_np.flatten(), bins=256, color='green', alpha=0.7)
        plt.title('Histogram of Redacted Image Pixel Values')
        plt.xlabel('Pixel Value')
        plt.ylabel('Frequency')
        plt.grid()


    return img_redacted, regions_after_filtering


names = []
sizes = []


scatter = go.Scatter(
    x=[], y=[], mode='markers+text',
    text=[], textposition='top center',
    marker=dict(size=10)
)

fig = go.FigureWidget(data=[scatter])
display(fig)

image_names = tqdm.tqdm(os.listdir(ECG_FOLDER), desc='Redacting images', unit='image')

for image_name in image_names:
    path = os.path.join(ECG_FOLDER, image_name)
    save_path = os.path.join(SAVE_FOLDER, image_name)

    print(image_name)
    if os.path.exists(save_path):
        continue


    redacted_image = None
    regions_after_filtering = None

    for r in REDACTED_AREA_PROPORTIONS:
        curr_redacted_image, curr_regions_after_filtering = redact_image(path, min_redacted_area_proportion=r, display=DISPLAY)
        max_area = max([a.area for a in regionprops(label(curr_regions_after_filtering > 0))])

        if max_area / curr_regions_after_filtering.size < 1.4e-2:
            redacted_image = curr_redacted_image
            regions_after_filtering = curr_regions_after_filtering
            break



    proportion_redacted = np.sum(regions_after_filtering > 0) / regions_after_filtering.size
    names.append(image_name)
    sizes.append(proportion_redacted)

    if proportion_redacted < 0.002:
        print(f'Low redaction proportion for {image_name}: {proportion_redacted:.4f}')

    Image.fromarray(redacted_image).save(save_path)

    with fig.batch_update():
        fig.data[0].x = np.zeros_like(sizes)
        fig.data[0].y = sizes
        fig.data[0].text = names



FigureWidget({
    'data': [{'marker': {'size': 10},
              'mode': 'markers+text',
              'text': [],
              'textposition': 'top center',
              'type': 'scatter',
              'uid': 'c8e16ca7-8dbe-498f-960e-c4d9972da7ef',
              'x': [],
              'y': []}],
    'layout': {'template': '...'}
})

Redacting images:   0%|          | 0/567 [00:00<?, ?image/s]

IMG_1735.JPG
IMG_2122.JPG
IMG_2152.JPG
IMG_2060.JPG
IMG_2056.JPG
IMG_2120.JPG
IMG_2062.JPG
IMG_2187.JPG
IMG_1975.JPG
IMG_2007.JPG
IMG_2014.JPG
IMG_1799.JPG
IMG_2072.JPG
IMG_1727.JPG
IMG_1760.JPG
IMG_2224.JPG
IMG_1776.JPG
IMG_2068.JPG
IMG_2044.JPG
IMG_2002.JPG
IMG_2027.JPG
IMG_2137.JPG
IMG_1932.JPG
IMG_1766.JPG
IMG_1859.JPG
IMG_2038.JPG
IMG_1726.JPG
IMG_1815.JPG
IMG_2140.JPG
IMG_1748.JPG
IMG_1989.JPG
IMG_2084.JPG
IMG_2097.JPG
IMG_2015.JPG
IMG_1883.JPG
IMG_1979.JPG
IMG_2244.JPG
IMG_2256.JPG
IMG_1721.JPG
IMG_1769.JPG
IMG_2077.JPG
IMG_2177.JPG
IMG_1913.JPG
IMG_2212.JPG
IMG_2079.JPG
IMG_1971.JPG
IMG_1923.JPG
IMG_1978.JPG
IMG_1955.JPG
IMG_2017.JPG
IMG_2003.JPG
IMG_2042.JPG
IMG_1838.JPG
IMG_1720.JPG
IMG_2115.JPG
IMG_2200.JPG
IMG_2180.JPG
IMG_2036.JPG
IMG_1855.JPG
IMG_1985.JPG
IMG_2155.JPG
IMG_2009.JPG
IMG_2130.JPG
IMG_2189.JPG
IMG_2190.JPG
IMG_1744.JPG
IMG_1861.JPG
IMG_1818.JPG
IMG_1839.JPG
IMG_1800.JPG
IMG_2195.JPG
IMG_1708.JPG
IMG_2243.JPG
IMG_2041.JPG
IMG_1691.JPG
IMG_2010.JPG
IMG_2184.JPG

Redacting images:  52%|█████▏    | 297/567 [00:04<00:04, 64.35image/s]

Proportion of redacted pixels: 0.0134798200325882, Max area: 329854.0, Threshold: 0.01
IMG_1881.JPG
80.39999999999999
Redacted threshold grayscale: 80.39999999999999
Proportion of redacted pixels: 0.0018256485600776258, Max area: 44674.0, Threshold: 0.013
IMG_2081.JPG
60.0
Redacted threshold grayscale: 60.0
Proportion of redacted pixels: 0.0029712456878176106, Max area: 72707.0, Threshold: 0.013
IMG_1758.JPG
82.8
Redacted threshold grayscale: 82.8
Proportion of redacted pixels: 0.002223847055161934, Max area: 54418.0, Threshold: 0.013
IMG_1988.JPG
48.0
Redacted threshold grayscale: 48.0
Proportion of redacted pixels: 0.002616201709441947, Max area: 64019.0, Threshold: 0.013
IMG_1927.JPG
97.2
Redacted threshold grayscale: 97.2
Proportion of redacted pixels: 0.0015816375569835779, Max area: 38703.0, Threshold: 0.013
IMG_1710.JPG
102.0
Redacted threshold grayscale: 102.0
Proportion of redacted pixels: 0.0016896464468140198, Max area: 41346.0, Threshold: 0.013
IMG_1870.JPG
67.2
Redacted th

...

Proportion of redacted pixels: 0.0024864929631983512, Max area: 60845.0, Threshold: 0.013
IMG_2032.JPG
76.8
Redacted threshold grayscale: 76.8


Redacting images: 100%|█████████▉| 566/567 [05:34<00:01,  1.22s/image]

Proportion of redacted pixels: 0.0025886580122244976, Max area: 63345.0, Threshold: 0.013
IMG_2227.JPG
68.39999999999999
Redacted threshold grayscale: 68.39999999999999


Redacting images: 100%|██████████| 567/567 [05:35<00:00,  1.69image/s]

Proportion of redacted pixels: 0.002555638268379247, Max area: 62537.0, Threshold: 0.013
